## 深层神经网络
### 经典损失函数

## 分类问题：交叉熵和softmax

> 交叉熵H(p,q)：刻画的是通过概率分布q来表达概率分布p的困难程度。这里p表示的是答案，q表示的是预测值。
交叉熵刻画的是两个概率分布的距离，也就是说交叉熵的值越小，两个概率分布越接近。


In [1]:
import tensorflow as tf

In [2]:
##tf.clip_by_value可以将一个张量中的数值限制在一个范围中,可以避免运算错误，比如log0是无效的
##这里设置下线2.5 上限4.5
with tf.Session() as sess:
    v = tf.constant([[1.0,2.0,3.0],[4.0,5.0,6.0]])
    print (tf.clip_by_value(v,2.5,4.5).eval())
    print (tf.reduce_mean(v).eval())

[[2.5 2.5 3. ]
 [4.  4.5 4.5]]
3.5


> 解决回归问题的神经网络一般只有一个节点，这个节点的输出值就是预测值。

## 回归问题：均方误差（MSE）

mse = tf.reduce_mean(tf.square(y_-y))#常用公式



## tf.where函数和tf.greater函数的方法

In [3]:
v1 = tf.constant([1.0,2.0,3.0,4.0])
v2 = tf.constant([4.0,3.0,2.0,1.0])

sess = tf.InteractiveSession()
print (tf.greater(v1,v2).eval())

print (tf.where(tf.greater(v1,v2),v1,v2).eval())
sess.close()

[False False  True  True]
[4. 3. 3. 4.]


## 实战

In [4]:
from numpy.random import RandomState
batch_size = 8

x = tf.placeholder(tf.float32,shape=(None,2),name='x-input')

#回归问题一般只有一个输出节点
y_ = tf.placeholder(tf.float32,shape=(None,1),name='y-input')

#定义一个单层的神经网络前向传播的过程，这里就是简单的加权和
w1 = tf.Variable(tf.random_normal([2,1],stddev=1,seed=1))
y = tf.matmul(x,w1)


#定义预测多了和预测少了的成本
loss_less = 10
loss_more = 1
loss = tf.reduce_sum(tf.where(tf.greater(y,y_),
                             (y - y_) * loss_more,
                             (y_- y) * loss_less))
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

#通过随机数生产一个模拟数据集
rdm = RandomState(1)
dataset_size = 128
X = rdm.rand(dataset_size,2)

# 设置回归的正确值为两个输入的和加上一个随机量。之所以加上一个随机量是为了
# 加入不可预测的噪音，否则不同损失函数的意义就不大了，因为不同存世函数都会在能
# 完全预测正确的时候最低。一般来说噪音为一个均值为0的小量，所以这里的噪音设置为
# -0.05~0.05的随机数
Y = [[x1 + x2 +rdm.rand()/10.0-0.05] for (x1,x2) in X]

# 训练神经网络
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    STEPS = 5000
    for i in range(STEPS):
        start = (i * batch_size) % dataset_size
        end = min(start+batch_size,dataset_size)
        sess.run(train_step,
                 feed_dict={x:X[start:end],y_:Y[start:end]})
    print (sess.run(w1))

[[1.0193471]
 [1.0428091]]


也就是说预测的函数是$1.0193471x_1+1.0428091x_2$。这要比$x_1+x_2$大，因为在损失函数中指定预测少了的损失更大(loss_less>loss_more)

> 梯度下降算法主要是用于**优化单个参数的取值**，而反向传播算法给出高效的方式在所有参数上使用梯度下降算法，从而使神经网络模型在训练数据上的损失函数尽可能的小。


反向传播算法是训练神经网络算法的核心算法

## 梯度下降算法的问题和解决方法

### 1.梯度下降不能保证全局最优解
只有当损失函数是凸函数时，梯度算法才能保证达到全局最优解。

### 2.梯度下降算法计算时间太长
要在全部训练数据上最小化损失，所以损失函数$J(\theta)$实在所有训练数据上的损失和.
> 解决方法，使用随机梯度下降。


随机梯度算法每次优化的是某一条数据上的损失函数，所以问题也十分明显:
**在某一条数据上损失函数更小并不能代表在全部数据上损失函数更小，于是使用随机梯度下降优化得到的神经网络甚至可能无法达到局部最优。**

### 真实情况，折中梯度下降算法和随机梯度下降算法
每次计算一小部分的训练数据的损失函数，这一小部分数据称为一个batch.
- **通过矩阵运算，每次在一个batch上优化神经网络的参数并不会比单个数据慢太多。**
- **每次使用一个batch可以大大减小收敛所需要的迭代次数，同时可以使收敛到的结果更加接近梯度下降的结果。**


## 神经网络进一步优化

### 学习率的设置
> 通过指数衰减的学习率既可以让模型在训练的前期快速接近较优解，又可以保证模型在训练后期不会有太大的波动，从而更加接近局部最优。


在训练神经网络的时候，需要设置学习率**控制参数更新的速度**

- 如果学习率为1，那么无论迭代多少轮，参数都会不停的摇摆。
- 如果学习率过小，那莪虽然能保证收敛，但是这回大大降低优化速度。

> 因此学习率不能太大，也不能太小。

解决方法：
- **使用tf.train.exponential_decay函数实现指数衰减学习率.**

代码实现了以下功能
```
decayed_learning_rate = learning_rate * decay_rate ^ (global_step /decay_steps)
```

- 其中decayed_learning_rate是每一轮优化时使用的学习率.
- 其中learning_rate为事先设定的**初始学习率**
- decay_rate为**衰减系数**
- decay_steps为**衰减速度**

一般来说初始学习率、衰减系数和衰减速度都是根据经验设置的，而且损失函数下降的速度和迭代结束之后总损失的大小没有必然联系。

## 过拟合问题
一般采用**正则化**。正则化的思想就是在损失函数中加入刻画模型复杂程度的指标。

分为L1正则化和L2正则化

L1:$$R(w)=||w||_1 = \sum|w_i|$$
L2:$$R(w)=||w||_2^2=\sum|w_i^2|$$

无论是哪种正则化方式，基本思想都是希望通过限制全中的大小，使得模型不能任意拟合训练数据中的随机噪音。
- L1正则化会让参数更加稀疏，但是L2不会。参数稀疏就是有更多的参数为0，这样可以达到类似特征选取的功能。
- L2不会稀疏的原因，参数会很小比如0.001，这个参数的平方基本忽略，于是模型不会进一步将参数调整为0
- L1正则化的计算公式不可导，L2正则化可导.

实践中也可以L1和L2正则化同时使用
$$R(w)=\sum{\alpha|w_i| + (1-\alpha)w_i^2}$$

In [14]:
##L1正则化样例
weights = tf.constant([[1.0,-2.0],[-3.0,4.0]])
with tf.Session() as sess:
    ##输出（|1|+|-2|+|-3|+|4|）*0.5=5
    print (sess.run(tf.contrib.layers.l1_regularizer(.5)(weights)))#L1
    ##输出(1^2+(-2)^2+(-3)^2+(4)^2)/2*0.5=7.5
    print (sess.run(tf.contrib.layers.l2_regularizer(.5)(weights))) #L2

5.0
7.5


上述方法，当神经网络的参数增多的时候，这样的方式导致损失函数loss定义很长，可读性差且容易出错。

> 这时候就需要用TensorFlow提供的collection集合。

以下通过集合计算一个5层神经网络带L2正则化的损失函数的计算方法

In [21]:
import tensorflow as tf

def get_weight(shape,y):
    #生成一个变量
    var = tf.Variable(tf.random_normal(shape),dtype=tf.float32)
    #add_to_collection将函数将这个新生成的变量L2正则化项加入集合。
    #这个函数的第一个参数'losses'是集合的名字，第二个参数是要加入这个集合的内容
    tf.add_to_collection(
        'losses',tf.contrib.layers.l2_regularizer(y)(var)
    )
    return var

x = tf.placeholder(tf.float32,shape=(None,2))
y_ = tf.placeholder(tf.float32,shape=(None,1))

batch_size = 8

#定义每一层网络中的节点数
layer_dimension = [2,10,10,10,1]

#神经网络层数
n_layers = len(layer_dimension)

#这个变量维护前向传播时最深层的节点，开始的时候就是输入层
cur_layer = x
#当前节点数
in_dimension = layer_dimension[0]

#for循环生成5层网络
for i in range(1,n_layers):
    #layer_dimension[i]为一下层的节点数
    out_dimension = layer_dimension[i]
    #生成当前层中的权重的变量，将L2加入到计算图集合中
    weight = get_weight([in_dimension,out_dimension],0.001)
    bias = tf.Variable(tf.constant(0.1,shape=[out_dimension]))
    #使用RELU激活函数
    cur_layer = tf.nn.relu(tf.matmul(cur_layer,weight) + bias)
    #进入下一层之前将一下层节点的个数更新为当前层节数的个数
    in_dimension = layer_dimension[i]
    
#在定义神经网络前向传播的同时已经将所有的L2正则化损失加入了图上的集合
#在这里只需要计算刻画模型在训练数据上表现的损失函数
mse_loss = tf.reduce_mean(tf.square(y_ - cur_layer))

#将军方误差损失函数加入到损失集合
tf.add_to_collection('losses',mse_loss)

#get_collection返回一个列表，这个列表是所有的这个集合中的元素，这个样例中
#这个元素就是损失函数的不同部分，将他们加起来就可以得到最终的损失函数
loss = tf.add_n(tf.get_collection('losses'))

In [27]:
loss.eval

<bound method Tensor.eval of <tf.Tensor 'AddN:0' shape=() dtype=float32>>

### 滑动平滑模型

采用梯度下降算法训练模型，使用滑动平均模型在很多应用中都可以一定程度上提高最终模型在测试数据上的表现。

tf.train.ExponentialMovingAverage来实现滑动平均模型

In [10]:
##滑动平滑模型使用
import tensorflow as tf 
#定义一个变量用来计算滑动平均，这个变量的初始值为9，，所有需要计算滑动变量必须是实数型
v1 = tf.Variable(0,dtype=tf.float32)
step = tf.Variable(0,trainable=False)

#定义一个滑动平均的类
ema = tf.train.ExponentialMovingAverage(0.99,step)
#定义一个更新变量滑动平均的操作。
maintrain_average_op = ema.apply([v1])

with tf.Session() as sess:
    #初始化所有变量
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    #获取滑动平均之后变量的取值
    print (sess.run([v1,ema.average(v1)]))
    
    #更新v1值到5
    sess.run(tf.assign(v1,5))
    sess.run(maintrain_average_op)
    print (sess.run([v1,ema.average(v1)]))
    
    ##更新到10000
    sess.run(tf.assign(step,10000))
    sess.run(tf.assign(v1,10))
    
    sess.run(maintrain_average_op)
    print (sess.run([v1,ema.average(v1)]))
    
    #再次更新滑动平均值，得到的新滑动平均值
    sess.run(maintrain_average_op)
    print (sess.run([v1,ema.average(v1)]))
    

[0.0, 0.0]
[5.0, 4.5]
[10.0, 4.555]
[10.0, 4.60945]
